In [ ]:
import os

os.chdir(fr'\\?\D:\Projects\Calm_ISO\5h_Injection\ISO_Total')

os.getcwd() 

In [ ]:
# creates a list of paths with labeled images
# mRNA_Red means the mRNA we are centering, mRNA_Green means the mRNA we are surrounding around centers (Red puncta)
import os

def get_labeled_overlap_path(mNRA_Red, mRNA_Green, SubSET=None):

    WORD1 = str(mNRA_Red).lower()
    WORD2 = str(mRNA_Green).lower()
    WORD3 = str(SubSET).lower()

    input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'

    labeled_path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                # Read the file content and convert it to lowercase
                with open(os.path.join(root, file), "r") as f:
                    file_content = f.read().lower()

                # Check if WORD1 and WORD2 and WORD3 are in the file content
                if WORD1 in file_content and WORD2 in file_content and WORD3 in file_content:
                # if WORD1 in file_content and WORD2 in file_content:
                    # Append the folder path to the list
                    labeled_path_list.append(root)

    # Return the list of paths
    return labeled_path_list



In [ ]:
labeled_puncta_path_list = get_labeled_puncta_path("RnaRyr2", "ProtRyr2", "WT")

len(labeled_puncta_path_list)
# labeled_puncta_path_list

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from skimage import io, util, morphology
import os
import re
import numpy as np
from skimage.measure import label, regionprops
import tifffile
import pandas as pd
import time
from sklearn.neighbors import NearestNeighbors
import math
import json
import pickle as pkl

input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'

os.chdir(input_dir)

def Puncta_Overlap_Processing(path_list, mRNA_Red, mRNA_Green, SubSet=None):
    
       
    # create a df to count red, green and overlaped centroids (puncta)
    df = pd.DataFrame()
    df_overlap_to_nuclei_distances = pd.DataFrame()
    df_overlap_to_nuclei_distances_normalized = pd.DataFrame()
    df_random_overlap_to_nuclei_distances_raw = pd.DataFrame()
    df_random_overlap_to_nuclei_distances_normalized = pd.DataFrame()
    
    WORD1 = str(mRNA_Red) # main centroid
    WORD2 = str(mRNA_Green) # secondary centroid
    WORD3 = str(SubSet)

    for path in path_list:
        min_distances = []  # list to store min distances
        min_distances_normalized = []  # list to store normalized min distances
        overlap_to_nuclei_distances = {}
        
        start_time = time.time()  # start the timer
 
        mRNA1_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Labeled_{WORD1}.tif')
        mRNA2_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Labeled_{WORD2}.tif') 
        # mRNA2_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{WORD2}_filtered_image.tif') 
        Body_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Body_image.tif')
        Nuclei_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Nuclei_image.tif')
        Cytosol_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Cytosol_image.tif')
        
        with open(os.path.join(path, os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()

        pattern = r"XYZCalibration\(um\/vox\):(\d+\.\d+),(\d+\.\d+),(\d+\.\d+):"
        match = re.search(pattern, text)

        if match:
            x_dim = float(match.group(1))
            y_dim = float(match.group(2))
            z_dim = float(match.group(3))
        else:
            print("No match found.")  

        voxel_dims = (z_dim, y_dim, x_dim)
        
        # Load the binary images
        red_img = tifffile.imread(mRNA1_Name_SubSET_mask_path) # Centroids with Zero coordinates
        green_img = tifffile.imread(mRNA2_Name_SubSET_mask_path) # Centroids with real coordinates
        cytosol = util.img_as_bool(tifffile.imread(Cytosol_SubSET_mask_path))
        nuclei_img = tifffile.imread(Nuclei_SubSET_mask_path)

        overlap_img = red_img*green_img*cytosol
        
        Body_bool = util.img_as_bool(tifffile.imread(Body_SubSET_mask_path))
        dilated_Body_bool = morphology.binary_dilation(Body_bool)
        edges_Body_bool = np.logical_xor(Body_bool, dilated_Body_bool)
        Nuclei_bool = util.img_as_bool(nuclei_img) 
        
        # Find the centroids of the red puncta
        red_labels = label(red_img)
        red_props = regionprops(red_labels)

        # Extract the centroids and store them in a list
        red_centroids = []
        for prop in red_props:
            z, y, x = prop.centroid
            centroid = (z * z_dim, y * y_dim, x * x_dim)
            red_centroids.append(centroid)

        # Find the centroids of the green puncta
        green_labels = label(green_img)
        green_props = regionprops(green_labels)

        # Extract the centroids and store them in a list
        green_centroids = []
        for prop in green_props:
            z, y, x = prop.centroid
            centroid = (z * z_dim, y * y_dim, x * x_dim)
            green_centroids.append(centroid)   
            
        # Find the centroids of the overlap puncta
        overlap_labels = label(overlap_img)
        overlap_props = regionprops(overlap_labels)

        # Extract the centroids and store them in a list
        overlap_centroids = []
        for prop in overlap_props:
            z, y, x = prop.centroid
            centroid = (z * z_dim, y * y_dim, x * x_dim)
            overlap_centroids.append(centroid)   
                
        # Get the coordinates of all 255 values in array1 and array2
        coords_Nuclei = np.argwhere(Nuclei_bool == True)
        coords_edges_Body = np.argwhere(edges_Body_bool == True)  
        
        # Scale the coordinates to physical units using the voxel dimensions
        coords_Nuclei_physical = coords_Nuclei * voxel_dims
        coords_edges_Body_physical = coords_edges_Body * voxel_dims
        
        # Initialize the NearestNeighbors object with n_neighbors=1 (find the nearest neighbor) 
        nn1 = NearestNeighbors(n_neighbors=1, algorithm='kd_tree', metric='euclidean')
        nn1.fit(coords_Nuclei_physical) #mRNA_Name to Nuclei fit
        distances, indices = nn1.kneighbors(coords_edges_Body_physical)
        temp_df3 = pd.DataFrame({'z': coords_edges_Body_physical[:, 0], 'y': coords_edges_Body_physical[:, 1], 'x': coords_edges_Body_physical[:, 2], 'distance': distances.flatten()}) # Create a pandas dataframe to store the results
        max_distance_from_nuclei = temp_df3['distance'].max()

        
        median_distance = 0
        median_distance_normalized = 0
 
        # Calculate distances from overlap centroids to the nearest nuclei pixel
        if overlap_centroids and len(coords_Nuclei_physical) > 0:
            overlap_centroids_array = np.array(overlap_centroids)
            nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(coords_Nuclei_physical)
            distancesNN, indices = nbrs.kneighbors(overlap_centroids_array)

            min_distances = [] # list to store min distances
            min_distances_normalized = [] # list to store min distances            
            
            for i, overlap_coord in enumerate(overlap_centroids):
                nuclei_coord = coords_Nuclei_physical[indices[i][0]]
                min_distance = distancesNN[i][0]
                overlap_to_nuclei_distances[overlap_coord] = {'nuclei_coordinates': tuple(nuclei_coord), 'distance': min_distance}

                min_distances.append(min_distance) # append min distance to the list
                min_distances_normalized.append(min_distance / max_distance_from_nuclei) # append min distance to the list

            # Calculate the median for min_distances and min_distances_normalized for the current path
            median_distance = np.quantile(min_distances, 0.5)
            median_distance_normalized = np.quantile(min_distances_normalized, 0.5)

            # create a DataFrame with min distances
            df_min_distances = pd.DataFrame(min_distances, columns=[os.path.basename(path)])
            df_overlap_to_nuclei_distances = pd.concat([df_overlap_to_nuclei_distances, df_min_distances], axis=1) # concatenate along the column
            
            df_min_distances_normalized = pd.DataFrame(min_distances_normalized , columns=[os.path.basename(path)])
            df_overlap_to_nuclei_distances_normalized = pd.concat([df_overlap_to_nuclei_distances_normalized, df_min_distances_normalized], axis=1) # concatenate along the column

        # Simulate random distribution of red centroids within the cytosol
        cytosol_coords = np.argwhere(cytosol == True)
        num_overlap_centroids = len(overlap_centroids)
        random_indices = np.random.choice(range(len(cytosol_coords)), num_overlap_centroids)
        random_cytosol_coords = cytosol_coords[random_indices]
        random_cytosol_coords_physical = random_cytosol_coords * voxel_dims
       
        # Calculate distances from random centroids to the nearest nuclei pixel
        if random_cytosol_coords_physical.size and len(coords_Nuclei_physical) > 0:
            nbrs_random = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(coords_Nuclei_physical)
            distances_random, indices_random = nbrs_random.kneighbors(random_cytosol_coords_physical)
            
            min_distances_random = distances_random.flatten()
            min_distances_random_normalized = min_distances_random / max_distance_from_nuclei
            min_distances_random_raw = min_distances_random
            
            # Calculate the median for min_distances_random_normalized for the current path
            median_distance_random_normalized = np.quantile(min_distances_random_normalized, 0.5)
            median_distances_random_raw = np.quantile(min_distances_random_raw, 0.5)
            
            # create a DataFrame with normalized random min distances
            df_min_distances_random_normalized = pd.DataFrame(min_distances_random_normalized, columns=[os.path.basename(path)])
            df_random_overlap_to_nuclei_distances_normalized = pd.concat([df_random_overlap_to_nuclei_distances_normalized, df_min_distances_random_normalized], axis=1)  # concatenate along the column
            df_min_distances_random_raw = pd.DataFrame(min_distances_random_raw, columns=[os.path.basename(path)])
            df_random_overlap_to_nuclei_distances_raw = pd.concat([df_random_overlap_to_nuclei_distances_raw, df_min_distances_random_raw], axis=1)  # concatenate along the column

        # Calculate and add the volume of the Body mask to df1
        puncta_red_total = len(red_centroids)
        puncta_green_total = len(green_centroids)
        puncta_overlap_total = len(overlap_centroids)
        percent_of_red_overlapped = puncta_overlap_total * 100 / puncta_red_total if puncta_red_total != 0 else 0
        percent_of_green_overlapped = puncta_overlap_total * 100 / puncta_green_total if puncta_green_total != 0 else 0
        
        voxel_volume = x_dim * y_dim * z_dim  # Calculate the volume of a single voxel
        cytosol_volume = np.sum(cytosol) * voxel_volume
        
        df_temp = pd.DataFrame({
            f'overlap_absolute_{WORD1}_{WORD2}_{WORD3}': [puncta_overlap_total],
            f'overlap_Normalized_{WORD1}_{WORD2}_{WORD3}': [puncta_overlap_total / cytosol_volume],
            f'Half_Dist_Membrane_distance_to_nuclei_{WORD2}': [temp_df3['distance'].quantile(0.5)],
            f'%of_{WORD1}_overlap_to_{WORD2}_{WORD3}': [percent_of_red_overlapped],
            f'%of_{WORD2}_overlap_to_{WORD1}_{WORD3}': [percent_of_green_overlapped],
            f'Half_Dist_Overlap_{WORD1}_{WORD2}_distance_to_nuclei_{WORD3}': [median_distance],
            f'Half_Dist_Overlap_{WORD1}_{WORD2}_distance_to_nuclei_{WORD3}_normalized': [median_distance_normalized],
            f'Half_Dist_Overlap_{WORD1}_{WORD2}_distance_to_nuclei_{WORD3}_raw': [median_distances_random_raw],            
            f'Half_Dist_Overlap_{WORD1}_{WORD2}_distance_to_nuclei_{WORD3}_normalized': [median_distance_random_normalized],

            "path": path
        })

        df = pd.concat([df, df_temp], ignore_index=True)
        
        end_time = time.time() # record end time
        elapsed_time = end_time - start_time # calculate elapsed timeCalm1
        print(f"Processed {file} in {elapsed_time:.2f} seconds")
    
    df.to_csv(f'Overlap_Analysis_{WORD1}_{WORD2}_{WORD3}.csv')       
    df_overlap_to_nuclei_distances.to_csv(fr'Overlap_{WORD1}_{WORD2}_to_Nuclei_Puncta_all_distances_{WORD3}.csv')     
    df_overlap_to_nuclei_distances_normalized.to_csv(fr'Overlap_{WORD1}_{WORD2}_to_Nuclei_Puncta_normalized_all_distances_{WORD3}.csv')     
    df_random_overlap_to_nuclei_distances_raw.to_csv(fr'random_{WORD1}_{WORD2}_to_Nuclei_Puncta_raw_distances_{WORD3}.csv')     
    df_random_overlap_to_nuclei_distances_normalized.to_csv(fr'random_{WORD1}_{WORD2}_to_Nuclei_Puncta_norm_distances_{WORD3}.csv')     
    
    return df, df_overlap_to_nuclei_distances, df_overlap_to_nuclei_distances_normalized

In [ ]:
labeled_overlap_path_list = get_labeled_overlap_path("Calm3", "Ryr2")
print(len(labeled_overlap_path_list))
Overlap_Calm3_Ryr2_Fig3, Overlap_Dist_Calm3_Ryr2_Fig3, Overlap_Dist_Norm_Calm3_Ryr2_Fig3 = Puncta_Overlap_Processing(labeled_overlap_path_list, "Calm3", "Ryr2", "Fig3")